# New York State Hospital Discharge Analysis

In [91]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [92]:
# 1) Load dataset (commas stripped automatically)
df = pd.read_csv(
    '/content/Hospital_Inpatient_Discharges__SPARCS_De-Identified___Cost_Transparency__Beginning_2009_20250426.csv',
    thousands=','
)
df.columns = df.columns.str.lower().str.replace(' ', '_')
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (1192827, 14)


,year,facility_id,facility_name,apr_drg_code,apr_severity_of_illness_code,apr_drg_description,apr_severity_of_illness_description,apr_medical_surgical_code,apr_medical_surgical_description,discharges,mean_charge,median_charge,mean_cost,median_cost
0,2016,4,Albany Memorial Hospital,194,1,Heart Failure,Minor,M,Medical,2,8375.41,8375.41,3585.05,3585.05
1,2016,4,Albany Memorial Hospital,194,2,Heart Failure,Moderate,M,Medical,40,14029.82,12176.95,6182.67,5253.15
2,2016,4,Albany Memorial Hospital,194,3,Heart Failure,Major,M,Medical,70,23921.77,20229.81,11149.49,9068.10
3,2016,4,Albany Memorial Hospital,194,4,Heart Failure,Extreme,M,Medical,12,51260.45,35210.82,26081.70,15230.62
4,2016,4,Albany Memorial Hospital,196,4,Cardiac Arrest,Extreme,M,Medical,1,25357.84,25357.84,7791.75,7791.75


In [93]:
# 2) Handle missing values
def handle_missing_values(df):
    df = df.copy()
    for c in df.select_dtypes(include=['float64','int64']).columns:
        df[c] = df[c].fillna(df[c].median())
    for c in df.select_dtypes(include=['object']).columns:
        df[c] = df[c].fillna(df[c].mode()[0])
    return df

df = handle_missing_values(df)

In [94]:
# 3) Encode categorical features
def encode_categorical(df):
    df = df.copy()
    encoders = {}
    for col in ['facility_name', 'apr_drg_code', 'apr_medical_surgical_code']:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        encoders[col] = le
    return df, encoders

df_encoded, label_encoders = encode_categorical(df)

In [106]:
# 4) Model 1: Hospital + DRG Predictions
def prepare_model1(df):
    dfc = df.copy()
    for col in ['discharges', 'median_cost', 'median_charge']:
        dfc[col] = pd.to_numeric(dfc[col], errors='coerce')
    m1 = (dfc
          .groupby(['facility_name','apr_drg_code','year'], as_index=False)
          .agg({
              'discharges':'sum',
              'median_cost':'mean',
              'median_charge':'mean'
          })
          .sort_values('year'))
    return m1

model1 = prepare_model1(df_encoded)
X1 = model1[['facility_name','apr_drg_code','year']]
y1_d = model1['discharges']
y1_c = model1['median_cost']
y1_ch = model1['median_charge']

# Split data
X1_tr, X1_te, y1_d_tr, y1_d_te = train_test_split(X1, y1_d, test_size=0.2, random_state=42, shuffle=False)
_, _, y1_c_tr, y1_c_te = train_test_split(X1, y1_c, test_size=0.2, random_state=42, shuffle=False)
_, _, y1_ch_tr, y1_ch_te = train_test_split(X1, y1_ch, test_size=0.2, random_state=42, shuffle=False)

def train_model(X_tr, y_tr, X_te, y_te, name):
    try:
        mdl = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    except:
        mdl = RandomForestRegressor(random_state=42)
    mdl.fit(X_tr, y_tr)
    preds = mdl.predict(X_te)
    mae = mean_absolute_error(y_te, preds)
    r2  = r2_score(y_te, preds)
    print(f"{name} → MAE: {mae:.2f}, R²: {r2:.2f}")
    return mdl

print("Training Model 1 - Discharges")
m1_d = train_model(X1_tr, y1_d_tr, X1_te, y1_d_te, 'Model1 - Discharges')

print("Training Model 1 - Median Cost")
m1_c = train_model(X1_tr, y1_c_tr, X1_te, y1_c_te, 'Model1 - Median Cost')

print("Training Model 1 - Median Charge")
m1_ch = train_model(X1_tr, y1_ch_tr, X1_te, y1_ch_te, 'Model1 - Median Charge')

Training Model 1 - Discharges
Model1 - Discharges → MAE: 43.61, R²: 0.60
Training Model 1 - Median Cost
Model1 - Median Cost → MAE: 10742.10, R²: 0.35
Training Model 1 - Median Charge
Model1 - Median Charge → MAE: 32402.78, R²: 0.45


In [96]:
# 5) Model 2: Total Discharges by DRG
def prepare_model2(df):
    return (df
            .groupby(['apr_drg_code','year'], as_index=False)['discharges']
            .sum()
            .sort_values('year'))

model2 = prepare_model2(df_encoded)
X2 = model2[['apr_drg_code','year']]
y2 = model2['discharges']
X2_tr, X2_te, y2_tr, y2_te = train_test_split(X2, y2, test_size=0.2, random_state=42, shuffle=False)

print("Training Model 2 - Total Discharges by DRG")
m2 = train_model(X2_tr, y2_tr, X2_te, y2_te, 'Model2 - Total Discharges by DRG')

Training Model 2 - Total Discharges by DRG
Model2 - Total Discharges by DRG → MAE: 3462.19, R²: 0.60


In [97]:
# 6) Model 3: Mean Cost Prediction
def prepare_model3(df):
    return df[['facility_name',
               'apr_drg_code',
               'apr_severity_of_illness_code',
               'year',
               'apr_medical_surgical_code',
               'mean_cost']].sort_values('year')

model3 = prepare_model3(df_encoded)
X3 = model3.drop('mean_cost', axis=1)
y3 = model3['mean_cost']
X3_tr, X3_te, y3_tr, y3_te = train_test_split(X3, y3, test_size=0.2, random_state=42, shuffle=False)

print("Training Model 3 - Mean Cost")
m3 = train_model(X3_tr, y3_tr, X3_te, y3_te, 'Model3 - Mean Cost')

Training Model 3 - Mean Cost
Model3 - Mean Cost → MAE: 11000.19, R²: 0.37


In [98]:
# Build your future DataFrame
future = (
    model1[['facility_name','apr_drg_code']]
    .drop_duplicates()
    .reset_index(drop=True)
)
future['year'] = 2025

# No label-encoding needed—these are already the same integer codes
X_future = future[['facility_name','apr_drg_code','year']]

# Call your models
future['predicted_discharges']    = m1_d.predict(X_future)
future['predicted_median_cost']   = m1_c.predict (X_future)
future['predicted_median_charge'] = m1_ch.predict(X_future)

print(future.head())

   facility_name  apr_drg_code  year  predicted_discharges  \
0             31           345  2025             -8.573933   
1            313           212  2025             17.851635   
2            313           211  2025             17.851635   
3            216            33  2025             17.588993   
4            216            32  2025             17.588993   

   predicted_median_cost  predicted_median_charge  
0            9602.712891             14210.911133  
1           22712.392578             87907.757812  
2           22712.392578             87907.757812  
3           10493.761719             36328.445312  
4            6430.768555             26471.236328  


In [99]:
# ---- Model 1: Discharges, Median Cost & Median Charge per Hospital+DRG ----
future1 = (
    model1[['facility_name','apr_drg_code']]
    .drop_duplicates()
    .reset_index(drop=True)
)
future1['year'] = 2025

# Predict & clean
X1_fut = future1[['facility_name','apr_drg_code','year']]

preds_d = m1_d.predict(X1_fut).clip(0, None).round().astype(int)
preds_c = m1_c.predict(X1_fut).clip(0, None).round().astype(int)
preds_ch= m1_ch.predict(X1_fut).clip(0, None).round().astype(int)

future1['predicted_discharges']       = preds_d
future1['predicted_median_cost']      = preds_c
future1['predicted_median_charge']    = preds_ch

# Inverse-transform codes back to strings
future1['facility_name_str'] = label_encoders['facility_name'].inverse_transform(future1['facility_name'])
future1['apr_drg_code_str']  = label_encoders['apr_drg_code'] .inverse_transform(future1['apr_drg_code'])

# Display
cols1 = [
    'facility_name_str','apr_drg_code_str','year',
    'predicted_discharges','predicted_median_cost','predicted_median_charge'
]
display(future1[cols1].head(10))

,facility_name_str,apr_drg_code_str,year,predicted_discharges,predicted_median_cost,predicted_median_charge
0,Cayuga Medical Center at Ithaca,930,2025,0,9603,14211
1,Westchester Medical Center,462,2025,18,22712,87908
2,Westchester Medical Center,461,2025,18,22712,87908
3,Putnam Hospital Center,55,2025,18,10494,36328
4,Putnam Hospital Center,54,2025,18,6431,26471
5,Putnam Hospital Center,53,2025,85,9298,31380
6,Cayuga Medical Center at Ithaca,912,2025,0,23521,45659
7,St. Joseph Hospital,43,2025,3,13465,42792
8,Medina Memorial Hospital,253,2025,53,5754,4968
9,Cayuga Medical Center at Ithaca,951,2025,19,17355,33459


In [100]:
# ---- Model 2: Total Discharges by DRG ----
future2 = (
    model2[['apr_drg_code']]
    .drop_duplicates()
    .reset_index(drop=True)
)
future2['year'] = 2025

X2_fut = future2[['apr_drg_code','year']]
future2['predicted_total_discharges'] = m2.predict(X2_fut).clip(0, None).round().astype(int)

# Inverse DRG code
future2['apr_drg_code_str'] = label_encoders['apr_drg_code'].inverse_transform(future2['apr_drg_code'])

# Display
cols2 = ['apr_drg_code_str','year','predicted_total_discharges']
display(future2[cols2].head(10))

,apr_drg_code_str,year,predicted_total_discharges
0,4,2025,3225
1,952,2025,1165
2,955,2025,1165
3,951,2025,8243
4,956,2025,1165
5,309,2025,4777
6,175,2025,19244
7,301,2025,21504
8,302,2025,35328
9,284,2025,21504


In [101]:
# ---- Model 3: Mean Cost Prediction ----
future3 = (
    model3[['facility_name','apr_drg_code','apr_severity_of_illness_code','apr_medical_surgical_code']]
    .drop_duplicates()
    .reset_index(drop=True)
)
future3['year'] = 2025

# Correct order for Model 3:
X3_fut = future3[[
    'facility_name',
    'apr_drg_code',
    'apr_severity_of_illness_code',
    'year',
    'apr_medical_surgical_code',
]]

future3['predicted_mean_cost'] = (
    m3
    .predict(X3_fut)
    .clip(0, None)
    .round()
    .astype(int)
)
# Inverse labels
future3['facility_name_str'] = label_encoders['facility_name'].inverse_transform(future3['facility_name'])
future3['apr_drg_code_str']  = label_encoders['apr_drg_code'] .inverse_transform(future3['apr_drg_code'])

# Display
cols3 = [
    'facility_name_str','apr_drg_code_str',
    'apr_severity_of_illness_code','apr_medical_surgical_code',
    'year','predicted_mean_cost'
]
display(future3[cols3].head(10))

,facility_name_str,apr_drg_code_str,apr_severity_of_illness_code,apr_medical_surgical_code,year,predicted_mean_cost
0,Wyoming County Community Hospital,844,3,0,2025,12618
1,Wyoming County Community Hospital,861,1,0,2025,3494
2,Wyoming County Community Hospital,861,2,0,2025,4827
3,Wyoming County Community Hospital,861,3,0,2025,7783
4,Wyoming County Community Hospital,930,2,0,2025,6602
5,Wyoming County Community Hospital,950,3,1,2025,30886
6,Wyoming County Community Hospital,950,4,1,2025,62288
7,Wyoming County Community Hospital,951,3,1,2025,18230
8,Wyoming County Community Hospital,811,1,0,2025,2056
9,Wyoming County Community Hospital,811,3,0,2025,3715


In [102]:
residuals = y_test - y_pred
n = len(residuals)
se  = residuals.std(ddof=1) / np.sqrt(n)             # standard error of the mean residual
moe = stats.t.ppf(0.975, df=n-1) * se                 # margin of error for 95% CI

print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R²:  {r2_score(y_test, y_pred):.2f}")
print(f"95% CI on the mean error: ±{moe:.2f}")

NameError: name 'y_test' is not defined